In [5]:
import cv2
import numpy as np

In [6]:
#提取前景
cap  = cv2.VideoCapture('vtest.avi')
backSub = cv2.createBackgroundSubtractorMOG2() #這行要放在while外面

while cap.isOpened():
    ret, frame = cap.read()
    fgmask = backSub.apply(frame)
    cv2.imshow('frame', fgmask)
    if cv2.waitKey(33)&0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [8]:
#去除陰影
cap  = cv2.VideoCapture('vtest.avi')
backSub = cv2.createBackgroundSubtractorMOG2()
while cap.isOpened():
    ret, frame = cap.read()
    fgmask = backSub.apply(frame)
    shadowval = backSub.getShadowValue()
    ret, nmask = cv2.threshold(fgmask, shadowval, 255, cv2.THRESH_BINARY)
    #nmask = cv2.GaussianBlur(nmask,(5,5),0)
    cv2.imshow("frame", nmask)
    if cv2.waitKey(33)&0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [9]:
#connected component
cap  = cv2.VideoCapture('vtest.avi')
backSub = cv2.createBackgroundSubtractorMOG2()


while cap.isOpened():
    arr = [0]*1000
    cnt = [0]*1000
    flag = -1
    ret, frame = cap.read()
    fgmask = backSub.apply(frame)
    shadowval = backSub.getShadowValue()
    ret, nmask = cv2.threshold(fgmask, shadowval, 255, cv2.THRESH_BINARY)
    for i in range(576):
        for j in range(768):
            if(nmask[i,j]!=0):
                if(i==0 and j ==0):
                    flag+=1
                    arr[flag] = flag+1
                    nmask[i,j] = arr[flag]
                    cnt[nmask[i,j]-1] +=1
                elif(i==0):
                    if(nmask[i,j-1]!=0):
                        nmask[i,j] = nmask[i,j-1]
                        cnt[nmask[i,j]-1] +=1
                    else:
                        flag+=1
                        arr[flag] = flag+1
                        nmask[i,j] = arr[flag]
                        cnt[nmask[i,j]] +=1
                elif(j==0):
                    if(nmask[i-1,j]!=0):
                        nmask[i,j] = nmask[i-1,j]
                        cnt[nmask[i,j]] +=1
                    else:
                        flag+=1
                        arr[flag] = flag+1
                        nmask[i,j] = arr[flag]
                        cnt[nmask[i,j]] +=1
                else:
                    
                    if(nmask[i-1,j]!=0 and nmask[i,j-1]==0):
                        nmask[i,j] = nmask[i-1,j]
                        cnt[nmask[i,j]-1] +=1
                    elif(nmask[i,j-1]!=0 and nmask[i-1,j]==0):
                        nmask[i,j] = nmask[i,j-1]
                        cnt[nmask[i,j]-1] +=1
                    elif(nmask[i-1,j] != 0 and nmask[i,j-1] != 0):       
                        nmask[i,j] = nmask[i-1,j]
                        arr[nmask[i,j-1]-1] = nmask[i,j]
                        cnt[nmask[i,j-1]-1] = cnt[nmask[i,j]-1] + cnt[flag] + 1
                        cnt[flag]=0
                    else:
                        flag+=1
                        arr[flag] = flag+1
                        nmask[i,j] = arr[flag]
                        cnt[nmask[i,j]-1] +=1
    cv2.imshow("frame", nmask)
    if cv2.waitKey(33)&0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

KeyboardInterrupt: 

In [28]:
cap  = cv2.VideoCapture('vtest.avi')
backSub = cv2.createBackgroundSubtractorMOG2()
while cap.isOpened():
    flag = 0
    label = []
    ret, frame = cap.read()
    fgmask = backSub.apply(frame)
    shadowval = backSub.getShadowValue()
    ret, nmask = cv2.threshold(fgmask, shadowval, 255, cv2.THRESH_BINARY)
    for i in range(576):
        for j in range(768):
            if(nmask[i][j]!=0):
                if(i==0 and j ==0):
                    flag+=1
                    nmask[i][j] = flag
                elif(i==0):
                    if(nmask[i][j-1]!=0):
                        nmask[i][j] = nmask[i][j-1]
                    else:
                        flag+=1
                        nmask[i][j] = flag
                elif(j==0):
                    if(nmask[i-1][j]!=0):
                        nmask[i][j] =nmask[i-1][j]
                    else:
                        flag+=1
                        nmask[i][j] =flag
                else:
                    if(nmask[i-1][j] != 0 and nmask[i][j-1] != 0 and nmask[i-1][j] != nmask[i][j-1]):
                        nmask[i][j] = min(nmask[i][j-1], nmask[i-1][j])
                        s = set([nmask[i][j-1], nmask[i-1][j]])
                        label.append(s)
                    elif(nmask[i-1][j] != 0 and nmask[i][j-1] != 0):
                        nmask[i][j] = nmask[i-1][j]
                    elif(nmask[i][j-1]!=0):
                        nmask[i][j] = nmask[i][j-1]
                    elif(nmask[i-1][j]!=0):
                        nmask[i][j] = nmask[i-1][j]

                    else:
                        flag+=1
                        nmask[i][j] = flag
    pool = set(map(frozenset, label))
    groups = []
    while pool:
        groups.append(set(pool.pop()))
        while True:
            for candidate in pool:
                if groups[-1] & candidate:
                    groups[-1] |= candidate
                    pool.remove(candidate)
                    break
            else:
                break
    for i in range(576):
        for j in range(768):
            if (nmask[i][j]!=0):
                for m in range(len(groups)):
                    if nmask[i][j] in groups[m]:
                        nmask[i][j] = min(groups[m])
    #cv2.imshow("frame", nmask)
    #if cv2.waitKey(33)&0xFF == ord('q'):
        #break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

[{128, 129, 122, 124, 125}, {35, 37, 38, 39, 31}, {66, 70, 71, 73, 74, 75, 76, 80, 90, 97, 100, 104, 106, 109, 110, 111, 112, 114, 117, 121, 126}, {96, 99, 103, 77, 78, 84, 87, 88, 89, 91, 92, 95}, {32, 34, 36, 30}, {120, 118, 119}, {51, 53, 54, 58}, {33, 29}, {98, 101, 102, 105, 107, 94}, {16, 19, 13}, {27, 28}, {45, 46}, {24, 21}]
[{122, 123, 125}, {65, 57, 54}, {129, 134}, {97, 99, 37, 102, 39, 104, 41, 42, 44, 79, 48, 113, 83}, {64, 96, 100, 75, 76, 77, 47, 81, 49, 85, 86, 90, 61, 94, 95}, {128, 133, 135, 138, 141}, {67, 63}, {80, 70, 71, 89}, {68, 36, 38, 107, 108, 110, 115, 88, 92, 30}, {105, 111, 103}, {18, 19}, {45, 46}, {12, 5}, {137, 131, 140}, {1, 2}, {73, 84}]
[{58, 135, 72, 140, 46, 111, 50, 52, 150, 55, 56, 154, 120}, {4, 6}, {68, 70}, {105, 115, 109}, {129, 163, 133, 45, 142, 143, 48, 144, 47, 51, 147, 153, 59, 126}, {130, 66, 73, 74, 106, 78, 86, 121, 91, 93}, {193, 195, 188}, {97, 76, 77, 146, 149, 57, 90, 125}, {88, 84}, {64, 67}, {162, 164}, {35, 36}, {180, 174}, {18

KeyboardInterrupt: 

In [6]:
flag = 1

cap  = cv2.VideoCapture('vtest.avi')
backSub = cv2.createBackgroundSubtractorMOG2()

while cap.isOpened():
    A=[0]*300
    ret, frame = cap.read()

    fgmask = backSub.apply(frame)
    shadowval = backSub.getShadowValue()
    ret, nmask = cv2.threshold(fgmask, shadowval, 255, cv2.THRESH_BINARY)
    for i in range(576):
        for j in range(768):
            if(nmask[i,j] != 0):
                if(i == 0 or j ==0):
                    if(nmask[i,j] != 0):
                        flag +=1
                        nmask[i,j] = flag
                elif (nmask[i-1,j]==0 and nmask[i,j-1] ==0):
                    flag +=1
                    nmask[i,j] = flag
                elif (nmask[i-1,j]!=0 and nmask[i,j-1]!=0 and nmask[i-1,j] != nmask[i,j-1]):
                    nmask[i,j] = nmask[i-1,j]
                    for k in range(i+1):
                        for l in range(768):
                            if (nmask[k,l] == nmask[i-1,j]):
                                nmask[k,l] = nmask[i,j-1]
                elif (nmask[i-1,j] != 0):
                    nmask[i,j] = nmask[i-1,j]
                elif (nmask[i,j-1] != 0):
                    nmask[i,j] = nmask[i,j-1]
    for i in range(576):
        for j in range(768):
            if nmask[i,j]!=0:
                A[nmask[i,j]]+=1
    for i in range(576):
        for j in range(768):
            if (A[nmask[i,j]]<1000):
                A[nmask[i,j]]-=1
                nmask[i,j]=0
    print(A)
    A=[0]*300
    flag =1
    #cv2.imshow("frame", nmask)
    if cv2.waitKey(33)&0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

[-440207, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-440917, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

KeyboardInterrupt: 